# Task 1 d, Calculating strength reserve factor of all elements

## Please enter your name (daniel, fabian, felix, yannis)

In [1]:
name = "yannis"

## Imports 

In [2]:
import pandas as pd
import sys 
import os

sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))

from formulas.columnbuckling import *
from formulas.panels import *
from formulas.strength import *
from formulas.helpers import *


In [3]:
# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])

## File Selection 

In [4]:
#Import the stress of the panels (OLD VERSION)
# paneldf = pd.read_excel(f'../data/{name}/panel.xlsx', index_col=0)

In [5]:
# This will become the new import for the panel stresses
paneldf_v2 = pd.read_csv(f'../data/{name}/panel_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

# Rename the element column to match
#element_component_df.rename(columns={'Element ID': 'element_id'}, inplace=True)
paneldf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
paneldf_v2 = add_component_names_to_elements(paneldf_v2, element_component_df)

paneldf_v2.rename(columns={'Loadcase': 'Load Case', 'XX': 'sigmaXX', 'YY': 'sigmaYY', 'XY': 'sigmaXY', 'ZZ': 'sigmaZZ', 'YZ': 'sigmaYZ', 'ZX': 'sigmaXZ'}, inplace=True)
paneldf_v2.drop(columns=['FileID', 'Step', 'Layer'], inplace=True)
paneldf_v2.reset_index(drop=True, inplace=True)
paneldf_v2.set_index('Element ID', inplace=True) # @yannisendr this is possible, since pandas allows non-unique indices, however operations that require an unique index won't work 

# Rename the load case column to match the old one
paneldf_v2['Load Case'] = paneldf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

# copy the new df to the previous one
paneldf = paneldf_v2
#print(paneldf.to_string())
paneldf_v2.head(10)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name
Element ID,,,,,
1,Subcase 1 (LC1),-65.783310,25.897854,12.579947,panel1
2,Subcase 1 (LC1),-81.867737,26.737642,20.594475,panel1
3,Subcase 1 (LC1),-66.729820,36.480675,20.465578,panel1
4,Subcase 1 (LC1),-96.377647,34.062004,2.383052,panel2
5,Subcase 1 (LC1),-88.315163,26.684858,25.010015,panel2
6,Subcase 1 (LC1),-95.011391,28.084337,26.246931,panel2
7,Subcase 1 (LC1),-92.893387,33.028202,-2.922334,panel3
8,Subcase 1 (LC1),-94.885368,26.438585,19.751455,panel3
9,Subcase 1 (LC1),-92.333893,29.367826,36.810879,panel3


In [6]:
# Import the stress of the stringers 
#stringerdf = pd.read_excel(f'../data/{name}/stringer.xlsx', index_col=0)

stringerdf_v2 = pd.read_csv(f'../data/{name}/stringer_v2.csv')
element_component_df = pd.read_csv('../data/element_component_matching.csv')

stringerdf_v2.rename(columns={'Elements': 'Element ID'}, inplace=True)

# now add the component column to the paneldf_v2 by matching the element_id
for index, row in stringerdf_v2.iterrows():
    element_id = row['Element ID']
    # Find the matching row in the element_component_df
    matching_row = element_component_df[element_component_df['Element ID'] == element_id]
    if not matching_row.empty:
        component = matching_row['Component Name'].values[0]
        stringerdf_v2.at[index, 'Component Name'] = component
    else:
        stringerdf_v2.at[index, 'Component Name'] = None  # or handle missing components as needed

stringerdf_v2.set_index('Element ID', inplace=True)

stringerdf_v2.rename(columns={'Element Stresses (1D):CBAR/CBEAM Axial Stress': 'sigmaXX', 'Loadcase': 'Load Case'}, inplace=True)
stringerdf_v2.drop(columns=['FileID', 'Step'], inplace=True)

stringerdf_v2['Load Case'] = stringerdf_v2['Load Case'].replace({
    1: 'Subcase 1 (LC1)',
    2: 'Subcase 2 (LC2)',
    3: 'Subcase 3 (LC3)',
})

#print(stringerdf.head(30))
stringerdf_v2.head(10)

stringerdf = stringerdf_v2


## Constants 

In [7]:
sigma_ult = 530

## Applying Functions 

In [8]:
#Adding reserve Factor column to the panels 
paneldf['reserveFactor'] = paneldf.apply(panelStrength_calc, sigma_ult= sigma_ult, axis=1)
paneldf.head(10)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name,reserveFactor
Element ID,,,,,,
1,Subcase 1 (LC1),-65.783310,25.897854,12.579947,panel1,4.128293
2,Subcase 1 (LC1),-81.867737,26.737642,20.594475,panel1,3.375476
3,Subcase 1 (LC1),-66.729820,36.480675,20.465578,panel1,3.493406
4,Subcase 1 (LC1),-96.377647,34.062004,2.383052,panel2,3.098383
5,Subcase 1 (LC1),-88.315163,26.684858,25.010015,panel2,3.126712
6,Subcase 1 (LC1),-95.011391,28.084337,26.246931,panel2,2.926613
7,Subcase 1 (LC1),-92.893387,33.028202,-2.922334,panel3,3.275139
8,Subcase 1 (LC1),-94.885368,26.438585,19.751455,panel3,3.056389
9,Subcase 1 (LC1),-92.333893,29.367826,36.810879,panel3,2.805038


In [9]:
#Adding reserve Factor column to the stringers 
stringerdf['reserveFactor'] = stringerdf.apply(stringerStrength_calc, sigma_ult= sigma_ult, axis=1)
stringerdf.head(10)

,Load Case,sigmaXX,Component Name,reserveFactor
Element ID,,,,
40,Subcase 1 (LC1),-76.643044,stringer1,4.610116
41,Subcase 1 (LC1),-69.669487,stringer1,5.071565
42,Subcase 1 (LC1),-78.626205,stringer1,4.493837
43,Subcase 1 (LC1),-80.222389,stringer2,4.404423
44,Subcase 1 (LC1),-84.115013,stringer2,4.200598
45,Subcase 1 (LC1),-84.032295,stringer2,4.204733
46,Subcase 1 (LC1),-80.344658,stringer3,4.397720
47,Subcase 1 (LC1),-82.797615,stringer3,4.267434
48,Subcase 1 (LC1),-84.042885,stringer3,4.204203


# Making things pretty 

## Delete unnecessary columns 

In [10]:
# First for the stringers 
stringerdf = stringerdf.drop(['Component Name','sigmaXX'], axis=1)
stringerdf

,Load Case,reserveFactor
Element ID,,
40,Subcase 1 (LC1),4.610116
41,Subcase 1 (LC1),5.071565
42,Subcase 1 (LC1),4.493837
43,Subcase 1 (LC1),4.404423
44,Subcase 1 (LC1),4.200598
...,...,...
62,Subcase 3 (LC3),12.644223
63,Subcase 3 (LC3),17.945101
64,Subcase 3 (LC3),11.878996


In [11]:
# Then also the panels 
paneldf=paneldf.drop(['Component Name', 'sigmaXX', 'sigmaYY', 'sigmaXY'], axis = 1)
paneldf.head(10)

,Load Case,reserveFactor
Element ID,,
1,Subcase 1 (LC1),4.128293
2,Subcase 1 (LC1),3.375476
3,Subcase 1 (LC1),3.493406
4,Subcase 1 (LC1),3.098383
5,Subcase 1 (LC1),3.126712
6,Subcase 1 (LC1),2.926613
7,Subcase 1 (LC1),3.275139
8,Subcase 1 (LC1),3.056389
9,Subcase 1 (LC1),2.805038


## Concat data frames beneth each other 

In [12]:
combined = pd.concat([paneldf,stringerdf],axis=0)
combined.sample(20)

,Load Case,reserveFactor
Element ID,,
49,Subcase 1 (LC1),5.157549
44,Subcase 2 (LC2),4.394614
47,Subcase 3 (LC3),12.845407
54,Subcase 1 (LC1),4.315360
15,Subcase 3 (LC3),0.884303
6,Subcase 1 (LC1),2.926613
51,Subcase 2 (LC2),3.863858
18,Subcase 1 (LC1),2.706090
15,Subcase 1 (LC1),2.313257


## Split into three seperate loadcase data frames 

In [13]:
loadCase1df = combined[combined['Load Case'] == 'Subcase 1 (LC1)']
loadCase2df = combined[combined['Load Case'] == 'Subcase 2 (LC2)']
loadCase3df = combined[combined['Load Case'] == 'Subcase 3 (LC3)']
loadCase1df.head(20)

,Load Case,reserveFactor
Element ID,,
1,Subcase 1 (LC1),4.128293
2,Subcase 1 (LC1),3.375476
3,Subcase 1 (LC1),3.493406
4,Subcase 1 (LC1),3.098383
5,Subcase 1 (LC1),3.126712
6,Subcase 1 (LC1),2.926613
7,Subcase 1 (LC1),3.275139
8,Subcase 1 (LC1),3.056389
9,Subcase 1 (LC1),2.805038


In [14]:
# Drop the loadcase columns 
loadCase1df = loadCase1df.drop(['Load Case'], axis = 1)
loadCase2df = loadCase2df.drop(['Load Case'], axis = 1)
loadCase3df = loadCase3df.drop(['Load Case'], axis = 1)
#Rename the reserveFactor column 
loadCase1df = loadCase1df.rename(columns={"reserveFactor": "Load Case 1 RF"})
loadCase2df = loadCase2df.rename(columns={"reserveFactor": "Load Case 2 RF"})
loadCase3df = loadCase3df.rename(columns={"reserveFactor": "Load Case 3 RF"})
loadCase2df.tail(5)

,Load Case 2 RF
Element ID,
62,4.394614
63,4.625367
64,4.120400
65,4.016318
66,4.217947


In [15]:
# Now we concat them back together 
outputdf = pd.concat([loadCase1df,loadCase2df, loadCase3df], axis=1)
outputdf

,Load Case 1 RF,Load Case 2 RF,Load Case 3 RF
Element ID,,,
1,4.128293,4.636882,1.658609
2,3.375476,3.979587,1.887084
3,3.493406,4.039977,1.592488
4,3.098383,3.472761,1.607341
5,3.126712,3.759171,1.885295
6,2.926613,3.408561,1.573809
7,3.275139,3.560277,1.565438
8,3.056389,3.555850,1.900337
9,2.805038,3.310751,1.485905


# ROUND & Export back to Excel 

In [16]:
outputdf = outputdf.round(rounding_digits)
outputdf.to_excel(f'../data/{name}/output/processed_d.xlsx')
outputdf.head(10)

,Load Case 1 RF,Load Case 2 RF,Load Case 3 RF
Element ID,,,
1,4.1283,4.6369,1.6586
2,3.3755,3.9796,1.8871
3,3.4934,4.0400,1.5925
4,3.0984,3.4728,1.6073
5,3.1267,3.7592,1.8853
6,2.9266,3.4086,1.5738
7,3.2751,3.5603,1.5654
8,3.0564,3.5558,1.9003
9,2.8050,3.3108,1.4859
